# Recurrent Neural Networks

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/inefficient-networks/blob/master/docs/notebooks/tensorflow/05-tensorflow-cnn.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/inefficient-networks?style=social)](https://github.com/particle1331/inefficient-networks)

---

## Introduction

Hello and welcome to recurrent neural networks.

So in this module, we're going to discuss recurrent neural networks and everything related to them,

as well as their variants such as LSD and Joyeux.

So first of all, we need to understand, why do we need Arnon's?

Why can't we just use normal feed forward networks?

And we're going to understand what our answer the general.

So usually Arnon's are used for time series analysis.

So when we have to make a prediction that requires remembering or it requires some memory, which is

based on previous data, not only the current data, then we make use of an Anadan.

So in a normal feed for that work, realize that we don't make use of previous data, we only make use

of the current inputs.

So Arnon's allow you to make use of previous data and to remember what happened in previous data.

So basically they implement a memory.

So they make use of sequential information, let me give you two examples to facilitate the explanation

of Arnon's stock price prediction is one example so we can predict the price of the current month by

just looking at one inputs, for example, the previous day.

What we need to do is we need to observe the data and the trend previously and in previous days and

previous months even in order to make a new prediction.

So this requires remembering what happened previously in order to predict the current and which is the

current price for the day.

Another example is text generation so we can predict the next word in a sentence based on the previous

word only.

We need to understand the general context of the sentence.

We do know what happened previously, the sentence in order to predict the next word.

So we need to look at the whole sentence to understand its context.

If you're predicting each word in a sentence.

So let's say the sentence is she spoke to her husband and you're to due to this location right here

and you want to put the word her, of course, you can't predict the words her only by looking at two.

So you're going to look back and realize what this sentence is about.

In this case, it's she spoke to.

And then once you understand this whole sorry, the previous words in the sentence and especially the

word she because she will allow you to predict her, then you can predict this this word right here.

So you need to understand the general context in this case.

It's she spoke to.

So the context here is that somebody spoke to and you need to also look back at the word she to understand

that the person that spoke is a female.

OK, so in this case, you can predict the words here.

So this is an example of using Arnett's.

Another way to think about ordinances is that they have a memory which captures information about what

has been calculated so far.

So in this case, the hidden state where she spoke and to have been calculated and now we can predict

the word her and.

Another thing is that recurrent neural networks are called recurrent because they perform the same task

for every element of a sequence.

So another way to think about Arnon's, as I told you, is that they have a memory which which makes

use of previous information and whatever has been calculated up until the point.

Now, an answer is very simply several copies of the same neural network that are aligned together and

each one passes its outputs to the next one.

So the first neural network passes the output.

The second neural network on the second network passes its output to the third neural network and so

on.

So neural network is called a timestep.

OK, so let's say that you have five copies of the neural network, then you have five time steps.

In this case, if I give you an example, then let's say we have we have a sequence and we care.

We care about three words of the sentence or sequence.

They'll say we want to encode these three words or predict them.

And then what we have is three copies of the neural network and each copy is called a timestep.

OK, so one time for each word, each word, each time step, which means each neural network receives

one of these three words.

So each copy, it's called the Times because it receives different inputs at different times.

So we're going to see later on, that's the Arnon's not only receive the previous sorry, the current

input, it also receives the previous hidden states.

And now we're going to explain what is a hidden state.

So that's why it's called the timestamp, because it receives different inputs at different times.

So if we have three words, then each time step, each neural network receives one of these words.

That's why it's called the timestep.



In [ ]:
import random
import warnings
from pathlib import Path

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline

import tensorflow as tf
import tensorflow.keras as kr
from tensorflow.keras.datasets import cifar10


DATASET_DIR = Path("./data").absolute()

warnings.simplefilter(action="once")
backend_inline.set_matplotlib_formats('svg')

seed = 0
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

Now, let's have a look at the equations for an answer.

So, as I told you, it's just a copy of Neural Network several times.

And here we have four copies.

So we have four time steps.

And now let's have a look at the equations for an hour and now realize that each of these five steps

here is just a neural network.

So we have a neural network which consists of the input layer, the hidden layer and the outputs layer,

the very simple neural network.

Now, what we're going to do is pass the first inputs and get the first input to the employer and then

pass the employer of the hidden layer.

And now what we're going to do, we're going to take this output of the hidden layer and pass it to

the next neural network.

So this next timestep or this next neural network receives the previous output of the hidden state as

well as the current inputs.

OK, and again, you do the same thing.

So you feed the current inputs, you get the representation, and now you feed the hidden representation

to the third neural network third times.

So the third time surpasses the previous state as well as the current inputs.

And you continue on up until the end of the sequence.

So what are the equations in this case?

We're going to project each of X and the hidden states and the first step at zero is just a sensor or

a matrix of zeros.

So this is how we initialize the hidden state since we don't have anything.

So we just initialize it to Jerome.

So what we're going to do is do two linear projections.

So realize they have different weights here.

The first projection is for the inputs, which is this right here, and the second projection is for

the hidden state.

And now we're going to use the same exact weights.

Remember, we're doing these are all just copies of the network.

So they're not different networks.

They're the same exact network working together, aligning together.

So they have all the same weights right here.

They just receive different.

And so we're going to take the the second input, again, projected through W X and we're going to take

the previous domestic and also projected through W each.

And the activation function here is the Dunwich activation function that is used an ordinance and now

we finish with our second that we're going to do the third time step again, Project X three and it's

two.

We're going to protect the privacy of the states and we're going to attach activation function we get

at three and now finally we take X3.

So we're going to pass it on the fourth time.

So we're going to projected through A, through W, X and we're going to take the fourth and what's

projected through W X running through a kind of activation function and we get Atwar.

And now if we're doing some classification or prediction task, then of course we need an output layer.

So in this case, we have an upper tier, which basically is a linear layer followed by a softmax in

order to classify our input.

So let's say we're doing text generation, then we're going to classify according to how many words

we have in our vocabulary.

So how what are the what is the word that we want to predict?

So this is a classification task.

So here, for example, I'm just making it very simple.

We have two words to predict.

So we want to predict whether it's the first word or the second word.

So we do this usual as we do a normal fit for the words.

We project the output, each one through a linear layer, which is called w w why in this case.

And we take this softmax and we get our prediction.

So this is nothing new here.

This is just the output.

And the most important thing to realize is that all of these steps share the same weight.

So they all have W, W, X and Y, they just receive difference.

And so they're working all together in order to understand the sequence.

Now, let's talk about how we can turn feed forward networks into Arnon's.

OK, now let's talk about how to visualize feed for networks and relationships to recurrent neural networks.

So we have our feet forward here.

This is the fourth neural network that we usually work with.

It consists of an input layer and outputs.

And now if we take this and flip it and then we just back it up so easily, we're just going to back

up all the neurons in each layer.

So the first layer has four neurons, the first layer has three neurons and the first layer has two

neurons.

And then we're just going to represent each of these multidimensional neurons in one neuron, OK?

So here we're going to represent an aunt and this is a representation of an aunt.

And why I'm doing this is because you're going to see this in many literature literature.

So many of these scientific papers represents an art and in this form right here.

But you should understand what they mean is not only one you're on.

This is actually a multidimensional neuron.

So here we have four neurons.

As you can see here, we have three neurons and here we have two.

They're just representing these multidimensional neurons into one.

You're on for simplicity.

So you should understand this because you're going to see this and many literature.

So this is our.

And remember, this is a multidimensional representation.

So we're representing a multidimensional, multidimensional neurons and just one neuron.

And now we're going to take this and copy it.

So let's say that we want five, five steps.

So we're going to copy this network five times.

So we're going to copy the same network for the number of times.

That's OK.

And this is how you presenting art.

And so this is how you're going to see and are represented in literature.

OK, so remember, this is the same exact network, they all share the same ways, they just receive

different inputs.

So here we have the same network copied four, five times, which is the number of five steps.

And again, remember, each of these is just this feeds forward neural network right here, OK?

And again, I emphasize that each of these neurons is multidimensional.

So this is what we have right here.

This is an example of three steps.

This is what you usually have.

So we have a feed forward that we're copied for three times, and each of these receive the output of

the previous neural network.

OK, so these are the hidden units in an urn.

So if you go back here, then these are basically the hidden neurons right here.

We just have them up for visualisations.

And each of these are cold attacks.

So this is the input layer, the layer upon layer.

So the first neuron receives the input of the first input of the first step and a vector of zeros since

we're initializing the hidden state and then it generates an output representation of the hidden layer.

So we're going to take the output of the hillier and pass it to the next timestep.

So the next time step or the next neural network receives the second input and the previous hidden state.

And then we were going to pass it to the third time step, so the third time step received the third

and fourth and the previous events and so on.

OK, so that's it for our own ends.

And the next video we'll be discussing about back propagation through time.

Thank you very much.

And I'll see you next time.